In [1]:
import pandas as pd
import numpy as np


In [2]:
xlsx = pd.ExcelFile('data/dataset.xlsx')
Matches = pd.read_excel(xlsx, 'Calisma_4',dtype={'Teams': str, 'League': str })
Perf_Overall = pd.read_excel(xlsx, 'Perf_Overall',dtype={'Teams': str, 'League': str})
Perf_Def=pd.read_excel(xlsx, 'Perf_Def',dtype={'Teams': str, 'League': str})
Perf_Off=pd.read_excel(xlsx, 'Perf_Off',dtype={'Teams': str, 'League': str})
Perf_Detailed=pd.read_excel(xlsx, 'Perf_Detailed')
Marketcap=pd.read_excel(xlsx, 'marketcap_new')

In [3]:
Matches.importance1.fillna(35,inplace=True)
Matches.importance2.fillna(35,inplace=True)
Matches.head()

,Date,League_ID,League,team1,team2,spi_perc,spi_1,spi_2,prob_1,prob_2,prob_0,importance1,importance2,scor_diff
0,2022-10-31,1854,Serie A,Verona,Roma,0.79,58.27,73.95,0.2429,0.4968,0.2604,31.5,60.1,-2
1,2022-10-31,1854,Serie A,Monza,Bologna,0.96,52.43,54.39,0.3983,0.3232,0.2785,35.2,31.3,-1
2,2022-10-31,1869,LaLiga,Elche,Getafe,0.84,52.69,62.53,0.3575,0.3370,0.3055,44.5,38.7,-1
3,2022-10-30,1854,Serie A,Empoli,Atalanta,0.68,48.00,70.95,0.1962,0.5713,0.2325,38.5,61.4,-2
4,2022-10-30,1843,Ligue 1,Auxerre,AC Ajaccio,0.88,46.20,52.39,0.3827,0.3305,0.2868,43.3,50.2,1


In [12]:
Perf_Overall.head()

,Teams,League,Goals,Shots_pg,Discipline,Possession,Pass,AerialsWon,Rating_Overall
0,Bayern Munich,Bundesliga,49,20.5,181,66.4,87.1,12.1,7.03
1,Paris Saint-Germain,Ligue 1,43,15.9,271,60.4,90.5,6.9,7.02
2,Manchester City,Premier League,40,17.6,111,66.6,89.4,13.4,6.96
3,Barcelona,LaLiga,33,16.9,313,64.9,88.8,10.4,6.93
4,Napoli,Serie A,37,18.1,200,60.0,86.8,12.9,6.86


In [15]:
Perf_Off.head()

,Teams,League,Shots_off,Shots_OT,Dribbles,Fouled,Rating_Off
0,Paris Saint-Germain,Ligue 1,16.4,7.6,11.7,12.6,7.00
1,Angers,Ligue 1,10.6,4.1,10.1,11.0,6.41
2,Bayern Munich,Bundesliga,20.4,8.9,9.9,10.1,7.03
3,Lyon,Ligue 1,13.7,5.7,9.7,10.7,6.71
4,Barcelona,LaLiga,16.9,6.4,9.6,11.4,6.93


In [14]:
Marketcap.head()

,Teams,League,Avg. Age,Total marketcap,Value per year
0,Manchester City,Premier League,27.7,1090000000,0.026403
1,Manchester United,Premier League,26.7,752000000,-0.052317
2,Chelsea,Premier League,27.2,846000000,0.039190
3,Paris Saint-Germain,Ligue 1,25.9,889000000,0.072603
4,Liverpool,Premier League,27.3,867000000,0.099950


### MERGING Perf_Def - Perf_Off - Perf_Overall - Perf_Detailed and MARKETCAP Datasets

In [4]:
# MERGING Perf_Def - Perf_Off - Perf_Overall - Perf_Detailed Datasets

step1=pd.merge(Perf_Def, Perf_Off, on=['Teams','League'], how='outer')
step2=pd.merge(step1, Perf_Detailed, on=['Teams','League'], how='outer')
step3=pd.merge(step2, Perf_Overall, on=['Teams','League'], how='outer')
step4=pd.merge(step3, Marketcap, on=['Teams','League'], how='outer')

### FILLING NAN VALUES AFTER MERGING

In [5]:
# FILLING NAN VALUES AFTER MERGING

columns_to_fill = ['OutOfBox', 'SixYardBox', 'PenaltyArea', 'Shots_def', 'Shots_off', 'Tackles', 'Interceptions', 'Fouls', 'Offsides', 'Shots_OT', 'Dribbles', 'Fouled', 'Goals', 'Shots_pg', 'Discipline', 'Possession', 'Pass', 'AerialsWon', "Avg. Age","Total marketcap","Value per year" ]

fill_values = [4, 1, 6, 14, 11, 18, 9, 12, 2, 4, 6, 10, 16, 12, 300, 50, 80, 13, 25.55 , 50000000, 0.32]

for i, col in enumerate(columns_to_fill):
    step4[col].fillna(fill_values[i], inplace=True)


In [6]:
step5=step4.dropna(axis=1)

In [7]:
step6= step5.rename(columns={'Total marketcap': 'Total_marketcap', 'Avg. Age': 'Avg_Age','Value per year': 'Value_per_year'})

## TOPSIS FUNCTION

In [8]:
 
## 5 FUNCTIONS TO CALCULATE THE CLOSENESS BASED ON GIVEN WEIGHTS

def normalize(decision_matrix):
    """Normalize the decision matrix."""
    norm = np.linalg.norm(decision_matrix, axis=0)
    return decision_matrix / norm

def weighted_decision_matrix(normalized_matrix, weights):
    """Calculate the weighted decision matrix."""
    return normalized_matrix * weights

def ideal_and_negative_ideal_solutions(weighted_matrix):
    """Determine the ideal and negative-ideal solutions."""
    ideal = np.max(weighted_matrix, axis=0)
    negative_ideal = np.min(weighted_matrix, axis=0)
    return ideal, negative_ideal

def separation_measures(matrix, ideal, negative_ideal):
    """Calculate the separation measures."""
    separation_ideal = np.linalg.norm(matrix - ideal, axis=1)
    separation_negative_ideal = np.linalg.norm(matrix - negative_ideal, axis=1)
    return separation_ideal, separation_negative_ideal

def relative_closeness(ideal_separation, negative_ideal_separation):
    """Calculate the relative closeness."""
    return negative_ideal_separation / (ideal_separation + negative_ideal_separation)

def topsis(data, weights):
    normalized_matrix = normalize(data)
    weighted_matrix = weighted_decision_matrix(normalized_matrix, weights)
    ideal, negative_ideal = ideal_and_negative_ideal_solutions(weighted_matrix)
    ideal_separation, negative_ideal_separation = separation_measures(weighted_matrix, ideal, negative_ideal)
    closeness = relative_closeness(ideal_separation, negative_ideal_separation)
    
    return closeness

### CALLING THE FUNCTIONS FOR EACH SECTION
####     DEFENCIVE - OFFENSIVE - OVERALL - DETAILED

In [10]:
# DEFENCIVE DATASET:
# NOTE: Negative values are bad for performance

# Criteria Weights
weights_def = np.array([-7, 5, 7, -3, -1])

# Selecting the relevant columns from the dataset
criteria_data_def = step6[['Shots_def', 'Tackles', 'Interceptions', 'Fouls', 'Offsides']].to_numpy()

# Applying TOPSIS to the dataset
closeness_def = topsis(criteria_data_def, weights_def)

# Adding the closeness scores to the dataset
step6['DEF_spi'] = closeness_def

In [11]:
# OFFENSIVE DATASET:
# NOTE: Negative values are bad for performance

# Criteria Weights
weights_off = np.array([5, 9, 3, -3])

# Selecting the relevant columns from the dataset
criteria_data_off = step6[['Shots_off', 'Shots_OT', 'Dribbles', 'Fouled']].to_numpy()

# Applying TOPSIS to the dataset
closeness_off = topsis(criteria_data_off, weights_off)

# Adding the closeness scores to the dataset
step6['OFF_spi'] = closeness_off

In [12]:
# OVERALL DATASET:
# NOTE: Negative values are bad for performance

# Criteria Weights
weights_overall= np.array([9, 7, -5, 5, 5, 3])

# Selecting the relevant columns from the dataset
criteria_data_overall = step6[['Goals', 'Shots_pg', 'Discipline', 'Possession', 'Pass', 'AerialsWon']].to_numpy()

# Applying TOPSIS to the dataset
closeness_overall = topsis(criteria_data_overall, weights_overall)

# Adding the closeness scores to the dataset
step6['OVRL_spi'] = closeness_overall

In [13]:
# DETAILED DATASET:
# NOTE: Negative values are bad for performance

# Criteria Weights
weights_detailed= np.array([5, 7, 9])

# Selecting the relevant columns from the dataset
criteria_data_detailed = step6[['OutOfBox', 'SixYardBox', 'PenaltyArea']].to_numpy()

# Applying TOPSIS to the dataset
closeness_detailed = topsis(criteria_data_detailed, weights_detailed)

# Adding the closeness scores to the dataset
step6['DETAIL_spi'] = closeness_detailed

#### FILTER THE DATASET WITH NECESSERY COLUMNS

In [14]:
step7=step6[['Teams', 'League', 'DEF_spi', 'OFF_spi', 'OVRL_spi', 'DETAIL_spi', 'Avg_Age','Value_per_year',
             'Total_marketcap']]


### MERGE THE DERIVED DATASET WITH MATCHES DATASET TWICE
#### First Merge for the Home Team - Second Merge for the Away Team

In [15]:
step8=pd.merge(Matches, step7, left_on=['team1','League'],right_on=['Teams','League'],how='inner')
step9=pd.merge(step8, step7, left_on=['team2','League'],right_on=['Teams','League'],how='inner')

#### HANDLE THE DATETIME 

In [16]:
step9['Year']=step9['Date'].dt.year
step9['Month']=step9['Date'].dt.month
step9['Week_ofday']=step9['Date'].dt.dayofweek
step9.sort_values('Date',ascending=False,inplace=True)

In [17]:
step9=step9[['Date', 'Year', 'Month', 'Week_ofday', 'League_ID', 'League',
             'team1', 'team2', 'spi_1','spi_2', 'prob_1', 'prob_2', 'prob_0',
             'importance1', 'importance2','DEF_spi_x', 'OFF_spi_x', 
             'DETAIL_spi_x','OVRL_spi_x', 'Avg_Age_x', 'Total_marketcap_x',
             'Value_per_year_x', 'DEF_spi_y', 'OFF_spi_y','DETAIL_spi_y',
             'OVRL_spi_y', 'Avg_Age_y','Total_marketcap_y', 'Value_per_year_y',
             'scor_diff']]

In [18]:
step10=step9.copy()
step10.reset_index(drop=True,inplace=True)

### CALCULATE THE POINTS FOR EACH MATCH

In [19]:
# Add new row for points. 
# We added %50 more for the away team's Tie condition
# Also for the 'Away' team, we added 4 points instead of 3
def calculate_point(row):
    if row['scor_diff'] > 0:
        return (3, 0)
    elif row['scor_diff'] < 0:
        return (0, 4)
    else:
        return (1, 1.5)

### CALCULATION FUNCTION OF THE 5 ROLLING POINTS OF EACH TEAM ( HOME - AWAY )

In [20]:
# Calculate the summing point of each team's last 5 matches
# NOTE: 
# If team is 'home' team, function returns only 5 'home' matches' rolling sum
# If team is 'away' team, function returns only 5 'away' matches' rolling sum

def rolling_points_team1(row, df):
    team = row['team1']
    
    # Get the index of the current row
    current_index = row.name  
    # Get the next 5 rows with the given team and index greater than the current index
    team_rows = df[(df['team1'] == team) & (df.index > current_index)].iloc[:5]
    # Calculate the sum of next 5 matches' points
    rolling5_team1 = team_rows['point_team1'].sum()  
    return rolling5_team1

def rolling_points_team2(row, df):
    team = row['team2']
    # Get the index of the current row
    current_index = row.name  
    # Get the next 5 rows with the given team and index greater than the current index
    team_rows = df[(df['team2'] == team) & (df.index > current_index)].iloc[:5]
    # Calculate the sum of next 5 matches' points
    rolling5_team2 = team_rows['point_team2'].sum()  
    return rolling5_team2

In [21]:
# calculate_point
# CALL the *Function* 

step10[['point_team1', 'point_team2']] = step10.apply(calculate_point, axis=1, result_type='expand')


In [22]:
# rolling_points_team1 & rolling_points_team1
# CALL the *Function* 

step10['rolling5_team1'] = step10.apply(lambda row: rolling_points_team1(row, step10), axis=1)
step10['rolling5_team2'] = step10.apply(lambda row: rolling_points_team2(row, step10), axis=1)

In [26]:
# Organize the dataset where our target (scor_diff) in the last column
step10=step10[['Date','Year', 'Month', 'Week_ofday', 'League_ID', 'League', 'team1',
       'team2', 'spi_1', 'spi_2', 'prob_1', 'prob_2', 'prob_0', 'importance1',
       'importance2', 'DEF_spi_x', 'OFF_spi_x', 'DETAIL_spi_x', 'OVRL_spi_x',
       'Avg_Age_x', 'Total_marketcap_x', 'Value_per_year_x', 'DEF_spi_y',
       'OFF_spi_y', 'DETAIL_spi_y', 'OVRL_spi_y', 'Avg_Age_y',
       'Total_marketcap_y', 'Value_per_year_y', 'point_team1',
       'point_team2', 'rolling5_team1', 'rolling5_team2', 'scor_diff']]

# SAVE AS A CSV FILE:

In [27]:
step10.to_csv('data/football_step10.csv')